In [1]:
import pandas as pd
import numpy as np
import os, time,json
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['tmdb_api_results_2010.json',
 'tmdb_api_results_2011.json',
 'tmdb_api_results_2000.json',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2012.csv.gz',
 'tmdb_api_results_2001.json',
 'final_tmdb_data_2010.csv.gz',
 'title_basics.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 '.ipynb_checkpoints',
 'tmdb_api_results_2012.json',
 'title_akas.csv.gz',
 'titleR.csv.gz',
 'final_tmdb_data_2011.csv.gz']

In [2]:
YEARS_TO_GET = [2013,2014,2015]

In [3]:
basics= pd.read_csv('Data/title_basics.csv.gz')

In [4]:
errors = [ ]

In [5]:
!pip install tmdbsimple

In [6]:
with open('/Users/alisonwilliams/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

In [7]:
tmdb.API_KEY =  login['api-key']

In [8]:
movie = tmdb.Movies()

In [9]:
info = movie.info

In [10]:
def get_movie_with_rating(movie_id):
    """Adapted from source = https://github.com/celiao/tmdbsimple"""
    #Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    
    #save the info releases dictionaries
    info = movie.info()
    releases = movie.releases()
##save a "certification" key in info with the certification
    info ['certification'] = c['certification']
    return info

In [11]:
errors = [ ]

In [12]:
def write_json(new_data, filename):
    """Appends a list of records (new_data) to a json file(filename),
    Adapted from://www.geeksforgeeks.org/append-to-json-file-using-python/"""
    with open(filename,'r+') as file:
        #first we load existing file into a dict
        file_data = json.load(file)
        ##choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)
        # sets files current position at offset.
        file.seek(0)
        #convert back to json
        json.dump(file_data, file)